# Overview: For Pedro and Josh

From Kaggle: In this competition, hosted by Jane Street, you'll build a model using real-world data derived from production systems, which offers a glimpse into the daily challenges of successful trading. This challenge highlights the difficulties in modeling financial markets, including fat-tailed distributions, non-stationary time series, and sudden shifts in market behavior.

When approaching modeling problems in modern financial markets, there are many reasons to believe that the problems you are trying to solve are impossible. Even if you put aside the beliefs that the prices of financial instruments rationally reflect all available information, you’ll have to grapple with time series and distributions that have properties you don’t encounter in other sorts of modeling problems. Distributions can be famously fat-tailed, time series can be non-stationary, and data can generally fail to satisfy a lot of the underlying assumptions on which very successful statistical approaches rely. Layer on all of this the fact that the financial markets are ultimately a human endeavor involving a large number of individuals and institutions that are constantly changing with advances in technology and shifts in society, and responding to economic and geopolitical issues as they arise - and you can start to get a sense of the difficulties involved!

In this challenge, we ask you to build a model using real-world data derived from some of our production systems. This data gives a very close picture of some of the things we have to do every day to be successful at trading in modern financial markets. We’ve assembled a collection of features and responders related to markets where we run automated trading strategies and are concerned about having good underlying models. To balance crafting a challenging, relevant problem that ties into our business while respecting the proprietary and highly competitive nature of our trading, you will notice that we have anonymized and lightly obfuscated some of the features and responders we present in the data. These modifications don’t change the essence of the problem at hand, but they do allow us to give you a difficult task that meaningfully illustrates the work we do at Jane Street.

Jane Street has spent decades relentlessly innovating on all aspects of our trading, and building machine learning models to aid our decision-making. These models help us actively trade thousands of financial products each day across 200+ trading venues around the world. While this challenge only presents a tiny fraction of the quantitative problems Jane Streeters work on daily, we are very interested in seeing how the Kaggle community will approach this challenge, and in engaging with you about your solutions to the problem!

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import html


# Notes on imports
Jane Street uses polars instead of pandas, could be worth checking out. It's supposed to be faster.

In [3]:
file_path = '../train.parquet/partition_id='
for i in range(10):
    df = pd.read_parquet(file_path + str(i), engine='fastparquet')
    print(df.shape)

(1944210, 92)
(2804247, 92)
(3036873, 92)
(4016784, 92)
(5022952, 92)
(5348200, 92)
(6203912, 92)
(6335560, 92)
(6140024, 92)
(6274576, 92)


In [5]:
df = pd.read_parquet(file_path + '0', engine='fastparquet')

na_cols = 0
for col in df.columns:
    if df[col].isna().sum() > 0:
        

0
0
0
0
1944210
1944210
1944210
1944210
1944210
0
0
0
16980
0
0
0
0
0
0
54992
63
9232
59
59
0
1944210
0
0
0
0
1944210
1944210
0
0
0
1944210
21737
21737
0
0
0
0
0
324732
38328
97113
324732
38328
97113
166374
166374
87
0
0
293120
2290
64120
293120
2290
64120
59
59
21732
0
0
0
153999
133274
136458
166374
166374
0
0
0
0
0
0
21732
21732
16
16
0
0
0
0
0
0
0
0
0
0
0
